In [1]:
import sympy as sp
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import Math


plt.rcParams['font.size'] = 12
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times New Roman"]
plt.rcParams["mathtext.fontset"] = "stix"
plt.rcParams["axes.grid"] = True

Функции Крылова:

$$
\begin{align*}
K_1(x) &= \ch{x}\cos{x} \\
K_2(x) &= \frac{1}{2} \left(\ch{x}\sin{x} + \sh{x}\cos{x}\right) \\
K_3(x) &= \frac{1}{2} \sh{x}\sin{x} \\
K_4(x) &= \frac{1}{4} \left(\ch{x}\sin{x} - \sh{x}\cos{x}\right)
\end{align*}$$

In [2]:
λ, z, P, m, EJ, L = sp.symbols("lamda z P m EJ_x L", positive=True)

K1 = sp.cosh(λ) * sp.cos(λ)
K2 = (sp.cosh(λ) * sp.sin(λ) + sp.sinh(λ) * sp.cos(λ)) / 2
K3 = sp.sinh(λ) * sp.sin(λ) / 2
K4 = (sp.cosh(λ) * sp.sin(λ) - sp.sinh(λ) * sp.cos(λ)) / 4

Константы интегрирования, найденные методом Крамера из СЛАУ, сформулированной на основе граничных условий на правом торце:

$$
\begin{align*}
C_1 &= \frac{P}{4 m^3 EJ_x} \frac{K_2(\lambda)K_3(\lambda) - \left(1 + K_1(\lambda)\right)}{K_3^2(\lambda) - K_2(\lambda)K_4(\lambda)} \\
C_2 &= \frac{P}{4 m^3 EJ_x} \frac{\left(1 + K_1(\lambda)\right)K_3(\lambda) - K_2^2(\lambda)}{K_3^2(\lambda) - K_2(\lambda)K_4(\lambda)}
\end{align*}
$$

In [3]:
Δ = K3**2 - K2 * K4
Δ1 = P / (4 * m**3 * EJ) * (K2 * K3 - (1 + K1) * K4)
Δ2 = P / (4 * m**3 * EJ) * ((1 + K1) * K3 - K2**2)

C1 = (Δ1 / Δ).subs(m, λ / L).simplify()
C2 = (Δ2 / Δ).subs(m, λ / L).simplify()

In [4]:
display(Math(rf"C_1 = {sp.latex(C1.series(n=0, x=λ))}"))
display(Math(rf"C_2 = {sp.latex(C2.series(n=0, x=λ))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
v = C1 * K1.subs(λ, λ * z) + C2 * K2.subs(λ, λ * z) + P * L**3 / (λ**3 * EJ) * K4.subs(λ, λ * z)
v_fun = sp.lambdify((z, λ), v.subs({P : 1, L : 1, EJ : 1}), modules=["numpy"])

zz = np.linspace(0.0, 1.0, 1001)
λλ = [0.5, 2, 25]

In [6]:
%matplotlib widget
plt.ioff()

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6,3))

(line, ) = ax.plot(zz, v_fun(zz, λλ[1]), "k")
title = ax.set_title(f"$λ = {λλ[1]}$")
ax.set_xlabel(r"$\frac{z}{L}$")
ax.set_ylabel("$v(z)$")
ax.minorticks_on()
ax.grid(True, which="minor", color="gray", lw=0.5, alpha=0.4)
ax.grid(True, which="major", color="k", lw=0.5)

slider = widgets.FloatSlider(value=λλ[1], min=λλ[0], max=λλ[2], step=0.1,
                             description="λ", continuous_update=True)

def on_change(change):
    λi = change["new"]
    line.set_ydata(v_fun(zz, λi))
    title.set_text(f"λ = {λi:.2f}")
    ax.relim()
    ax.autoscale_view(scalex=False, scaley=True)
    fig.canvas.draw_idle()

slider.observe(on_change, names="value")
display(widgets.VBox([slider, fig.canvas]))